### Build and push image

In [10]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-tf-profiler

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)
#$(aws ecr get-login --region us-east-1 --no-include-email --registry-ids 763104351884)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}


docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon   21.5kB
Step 1/4 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.0-cpu
 ---> 63fa6a04e15e
Step 2/4 : RUN pip3 install numpy codeguru_profiler_agent flask
 ---> Using cache
 ---> a38c9fbe0b46
Step 3/4 : COPY Files/serve.py  /sagemaker/serve.py
 ---> Using cache
 ---> 142961ab6adc
Step 4/4 : COPY Files/gunicorn_conf.py /sagemaker/gunicorn_conf.py
 ---> Using cache
 ---> 874268ff3a81
Successfully built 874268ff3a81
Successfully tagged sagemaker-tf-profiler:latest
The push refers to repository [171503325295.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tf-profiler]
691593db6e1e: Preparing
f5003bb4df43: Preparing
3857443b672e: Preparing
391203f8b1d6: Preparing
00ba44ea47e2: Preparing
87f2086afef8: Preparing
6988657c7a0c: Preparing
4007fb38d4c2: Preparing
8747cacd7a48: Preparing
07d8bf252ada: Preparing
0f5aaaadabc2: Preparing
da2011aa4f9c: Preparing
9b348cdac0e6: Preparing
44a24723f338: Preparing
ce94cf557356: P

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [11]:
import boto3

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name
image_uri = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-tf-profiler:latest".format(account_id,region)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [12]:
import sagemaker

role = sagemaker.get_execution_role()

### Create Feature Group and ingest 
Below we generate some random input data for our model and ingest it into Feature Store.
We do this as our inference.py script will confirm that the input we are sending to our model for prediction is the latest features we saved in Feature Store

In [41]:
import pandas as pd
import numpy as np
from time import gmtime, strftime
import time


random_input = np.random.rand(1, 1, 3, 3)
random_input = random_input.tolist()
df = pd.DataFrame()

df["RandomInput"] = [random_input]
df = df.astype({'RandomInput': 'string'})

current_time_sec = int(round(time.time()))
df["EventTime"] = current_time_sec
df = df.astype({'EventTime': 'float64'})
df['Index'] = range(1, len(df) + 1)
df["RandomInput"]


0    [[[[0.9098874423843568, 0.8892154494013925, 0....
Name: RandomInput, dtype: string

In [42]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.session import Session
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

prefix = 'DEMO-CodeGuru-Profiler'
my_features_feature_group = FeatureGroup(
    name="my-features", sagemaker_session=sagemaker_session
)


my_features_feature_group.load_feature_definitions(data_frame=df)

record_identifier_feature_name="RandomInput"

my_features_feature_group.create(
    s3_uri=f"s3://{bucket}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

ResourceInUse: An error occurred (ResourceInUse) when calling the CreateFeatureGroup operation: Resource Already Exists: FeatureGroup with name my-features already exists. Choose a different name.

In [43]:
def check_feature_group_status(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group to be Created")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group.name} successfully created.")


check_feature_group_status(my_features_feature_group)

FeatureGroup my-features successfully created.


In [44]:
my_features_feature_group.ingest(data_frame=df, max_workers=3, wait=True)


IngestionManagerPandas(feature_group_name='my-features', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f2c3b7b9518>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f2c3b50a5c0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

### Create an inference.py
Here we create an inference.py with an expensive function that retrieves the latest feature value from Feature Store

In [46]:
%%writefile inference.py

import json
import time
import numpy as np
import logging
import boto3

client = boto3.client('sagemaker-featurestore-runtime')


def retrieve_latest_features_boto3_client_create(data,record_id):
    print("retrieve_latest_features_boto3_client_create")
    import boto3

    client = boto3.client('sagemaker-featurestore-runtime')

    response = client.get_record(
    FeatureGroupName='my-features',
    RecordIdentifierValueAsString=str(record_id),
    )
    
    return response["Record"][0]["ValueAsString"]
    
    
    
    
def retrieve_latest_features(data,record_id):
    print("retrieve_latest_features")
    response = client.get_record(
    FeatureGroupName='my-features',
    RecordIdentifierValueAsString=str(record_id),
    )
    
    return response["Record"][0]["ValueAsString"]
    

def input_handler(data, context):
    """ Pre-process request input before it is sent to TensorFlow Serving REST API
    Args:
        data (obj): the request data, in format of dict or string
        context (Context): an object containing request and configuration details
    Returns:
        (dict): a JSON-serializable dict that contains request body and headers
    """

    if context.request_content_type == 'application/json':

        d = data.read().decode('utf-8')
      
        input_data = json.loads(d)

        assert input_data["data"] == json.loads(retrieve_latest_features(input_data["data"],input_data["id"]))
      
        return json.dumps({"inputs" : input_data["data"]})


    raise ValueError('{{"error": "unsupported content type {}"}}'.format(
        context.request_content_type or "unknown"))


def output_handler(data, context):
    """Post-process TensorFlow Serving output before it is returned to the client.
    Args:
        data (obj): the TensorFlow serving response
        context (Context): an object containing request and configuration details
    Returns:
        (bytes, string): data to return to client, response content type
    """
    if data.status_code != 200:
        raise ValueError(data.content.decode('utf-8'))
    response_content_type = context.accept_header
    prediction = data.content
    return prediction, response_content_type

Overwriting inference.py


### Deploy our Model to an Endpoint
Our container has been pushed to ECR and our Model is in S3 now we have everything we need to Deploy to a SageMaker Endpoint.

In [47]:
%%time
saved_model = (
    "s3://sagemaker-sample-data-{}/tensorflow/model/resnet/resnet_50_v2_fp32_NCHW.tar.gz".format(
        region
    )
)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [49]:
from sagemaker.tensorflow.serving import Model
from sagemaker.local import LocalSession

tensorflow_model = Model(model_data=saved_model,
                         role=role,
                         entry_point = "inference.py",
                         image_uri=image_uri,
                         #env={"AWS_DEFAULT_REGION": "us-east-1"},
                       # sagemaker_session = LocalSession()
                        )

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [50]:
%%time
predictor = tensorflow_model.deploy(
    initial_instance_count=1, instance_type="ml.t2.medium"
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------!CPU times: user 8.07 s, sys: 929 ms, total: 9 s
Wall time: 3min 40s


In [ ]:
%%time
import numpy as np
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer

input_data = {
    "data": random_input,
    "id" : "1"
}
#for i in range (0,20000):
while True:
    #predictor.serializer = ser
    prediction = predictor.predict(input_data)
   # print(prediction)
    #break

    